# Stanza NER

[Stanza's](https://stanfordnlp.github.io/stanza/neural_pipeline.html) NER pipeline, like SpaCy's, is built on Ontonotes so it can recognize Work of Art.

It seems to do a worse job than `0022-spacy-ner` so we'd prefer that.
It's clear I should annotate some examples to automatically do these kinds of evaluations.

In [3]:
import stanza

import pandas as pd

In [4]:
book_recs = pd.read_csv('../data/02_intermediate/hn_ask_book_recommendations.csv')

In [2]:
nlp = stanza.Pipeline('en')

2022-06-27 20:43:35 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2022-06-27 20:43:36 INFO: Use device: gpu
2022-06-27 20:43:36 INFO: Loading: tokenize
2022-06-27 20:43:55 INFO: Loading: pos
2022-06-27 20:43:56 INFO: Loading: lemma
2022-06-27 20:43:56 INFO: Loading: depparse
2022-06-27 20:43:58 INFO: Loading: sentiment
2022-06-27 20:43:59 INFO: Loading: ner
2022-06-27 20:44:00 INFO: Done loading processors!


In [7]:
import re
import html

def clean(text):
    text = html.unescape(text)
    text = text.replace('<i>', '"')
    text = text.replace('</i>', '"')
    text = text.replace('<p>', '\n\n')
    text = re.sub('<a href="(.*?)".*?>.*?</a>', r'\1', text)
    return text

In [8]:
%time docs = [nlp(text) for text in book_recs.text.head(20).map(clean).to_list()]

CPU times: user 9.04 s, sys: 6.01 s, total: 15 s
Wall time: 15.4 s


Looking through this these are worse than SpaCy's.

In [13]:
doc = docs[0]
for doc in docs[:15]:
    print(doc.text)
    print()
    print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')
    print()

- Existential Rationalism: Handling Hume's Fork (second edition)
- Living with the Himalayan Masters
- The Outsider
- Hirohito: Behind the Myth

entity: Handling Hume's Fork	type: WORK_OF_ART
entity: second	type: ORDINAL
entity: Living with the Himalayan Masters
- The Outsider
- Hirohito: Behind the Myth	type: WORK_OF_ART

The Coming of Neo-Feudalism by Joel Kotkin

entity: The Coming of Neo	type: WORK_OF_ART
entity: Joel Kotkin	type: PERSON

Probably "Reaper", by Will Wight. It’s not an insightful nonfiction book or a piece of high literature, but the whole Cradle series is very, very fun.

entity: Will Wight	type: PERSON

A Gentleman in Moscow by Amor Towles. I spent a lot of the year in isolation, only seeing a few people and this book felt like an appropriate analogy. It was also very heartwarming when I really needed something to lift me up.

entity: Moscow	type: GPE
entity: Amor Towles	type: PERSON

"In Cold Blood" by Truman Capote. It's a masterpiece.

entity: Cold Blood"	type: 